In [1]:
%load_ext autoreload
%autoreload 2

In [1]:
import sys
import pyarrow as pa
import pyarrow.parquet as pq
import os
import pandas as pd

sys.path.insert(0, '../')
from src.mongo_utils import collection_de, create_set, get_all_entries, set_datasplit
from src.utils import sanitize_context

In [ ]:
filter_list = [
    "Holdung",
]
"Titel != das, context_word = das"

In [2]:
def generator(col, split, postprocess=lambda x, y, a, b: (x, y, a, b)):
    pointer = get_all_entries(col, split=split)
    buffer = []

    for data in pointer:
        if not buffer:
            buffer += create_set(data, replace_none=True)
        if len(buffer) == 0:
            continue
        yield postprocess(*buffer.pop(0))

In [3]:
test = generator(collection_de, split="test")
train = generator(collection_de, split="train")
val = generator(collection_de, split="val")

In [4]:
for i in train:
    if (i[0] == 'Stubenhocker'):
        print(i)

('Stubenhocker', 'Stubenhocker', "„Das Internet ist schon lange kein Platz für »Techies« und ''Stubenhocker'' mehr, die einsam und in sich gekehrt ihren individuellen Interessen nachgehen.“<ref>{{Literatur | Autor=Heiko Haller, Markus Hartwig, Arne Liedtke | Titel=Google Analytics und Co | TitelErg=Methoden der Webanalyse professionell anwenden| Verlag=Pearson | Jahr=2010 | Seiten=334| Online=Zitiert nach {{GBS|yMteJgL7tGMC|PA334|Hervorhebung=Stubenhocker}}}}.</ref>", '{{ugs.}}: Person, die sich vom sozialen Leben abkapselt bzw. ihr soziales Umfeld auf einen minimalen Raum beschränkt; Person, die ihre Stube nicht verlassen und ihre Ruhe haben will')
('Stubenhocker', 'Stubenhocker', "„Ich warte auf eine Kontaktanzeige, in der eine Frau einen maulfaulen ''Stubenhocker'' sucht, der weder tanzt noch wandert, aber gerne die Sopranos guckt und dabei Pralinen futtert.“<ref>Jan Weiler: ''Mein Leben als Mensch.'' Illustriert von Larissa Bertonasco. 2. Auflage. Rowohlt, Reinbek 2009, Seite 137. 

In [3]:
base_path = "../dataset/v4/"

In [4]:
split = "train"

def write_split(col, split, base_path=base_path, overwrite=False):
    if not os.path.exists(base_path):
        os.mkdir(base_path)
    path = os.path.join(base_path, f"{split}.parquet")
    if os.path.exists(path):
        if overwrite:
            os.remove(path)
        else:
            raise Exception("Already existing!")
    
    data = []
    for value in generator(col, split):
        data.append(value)
    df = pd.DataFrame(data=data, columns=["title", "context_word", "context_sentence", "gt"])
    table = pa.Table.from_pandas(df)
    pq.write_table(table, path)

In [5]:
write_split(collection_de, "train", overwrite=True)
write_split(collection_de, "test", overwrite=True)
write_split(collection_de, "val", overwrite=True)

In [203]:
train = generator(collection_de, split="train")

In [239]:
ex = next(train)
cl = lambda x, y, a, b: f"{sanitize_context(a)}\n\nWas ist die Definition von \"{y}\"?"
print(ex)
print()
print(cl(*ex))

('Januar', 'Januar,', "„Es war in der kalten Jahreszeit, genauer gesagt im ''Januar,'' als ich meinen Asylantrag in Deutschland stellte.“", 'erster, 31-tägiger Monat im Kalenderjahr')

Es war in der kalten Jahreszeit, genauer gesagt im Januar, als ich meinen Asylantrag in Deutschland stellte.

Was ist die Definition von "Januar,"?


In [2]:
all_data = list(get_all_entries(collection_de))

In [3]:
def count_set(data):
    sets = []
    for el in data:
        sets += create_set(el)
    return len(sets)

In [4]:
all_sets = []
for el in all_data:
    all_sets += create_set(el)

In [5]:
len(all_data)

129674

In [6]:
len(all_sets)

359515

In [80]:
len(all_sets)

413169

In [7]:
from sklearn.model_selection import train_test_split

train_val, test = train_test_split(all_data, test_size=0.1, random_state=42)
train, val = train_test_split(train_val, test_size=0.11, random_state=42)

In [8]:
test_count = count_set(test)
train_val_count = count_set(train_val)
train_count = count_set(train)
val_count = train_val_count - train_count
print(f"Test: {len(test)}, Train: {len(train)}, Val: {len(val)}, Train+Val: {len(train_val)}")
print(f"Test: {test_count}, Train: {train_count}, Val: {val_count}, Train+Val: {train_val_count}")

Test: 12968, Train: 103868, Val: 12838, Train+Val: 116706
Test: 35733, Train: 288119, Val: 35663, Train+Val: 323782


In [9]:
print(test_count / (test_count+train_val_count))
print(val_count / (test_count+train_val_count))
print(train_count / (test_count+train_val_count))

0.09939223676341738
0.09919753000570213
0.8014102332308805


In [10]:
print(len(test) / (len(test)+len(train_val)))
print(len(val) / (len(test)+len(train_val)))
print(len(train) / (len(test)+len(train_val)))

0.10000462698767679
0.09900211299103907
0.8009932600212841


In [11]:
set_datasplit(collection_de, test, 'test')
set_datasplit(collection_de, train, 'train')
set_datasplit(collection_de, val, 'val')

In [12]:
import pandas as pd

test_ = pd.DataFrame(test, columns=["title"])
test_

,title
0,Vollform
1,Neugeborener
2,Neun
3,Springerbauer
4,Achievementtest
...,...
12963,gegenstandslos
12964,Namenwort
12965,Fürstentum
12966,korrektiv


In [13]:
test[0]['_id']

ObjectId('6687babe9a5a7acaf44cfca0')

In [36]:
from src.mongo_utils import collection, db

In [37]:
collection

Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'wikipedia_dump'), 'articles_2')

In [20]:
test[0]["_id"]

ObjectId('6613c7782cfcd7a854112d94')

In [33]:
from pymongo import UpdateOne

def set_datasplit(col, dataset, split):
    updates = []
    for d in dataset:
        updates.append(UpdateOne({'_id': d['_id']}, {"$set": {"split": split}}))
    col.bulk_write(updates)
    
def unset_datasplit(col, dataset):
    updates = []
    for d in dataset:
        updates.append(UpdateOne({'_id': d['_id']}, {"$unset": {"split": 1}}))
    col.bulk_write(updates)

In [40]:
set_datasplit(val, 'val')

In [41]:
test_data = list(get_all_entries(split="test"))
len(test_data)

15588